In [5]:
import pandas as pd
import requests
import json


In [6]:
API_KEY = 'api-key' # st louis fed 
ENDPOINT = 'https://api.stlouisfed.org/fred/series/observations'

# ids
BS = 'WALCL'
TGA = 'WTREGEN'
REPO =  'RRPONTSYD'
SP500 = 'SP500'

In [7]:
#Fred
def pull_data(SERIES_ID):
  params = {
    "series_id": SERIES_ID,
    "api_key": API_KEY,
    "file_type": "json",
    }
  response = requests.get(ENDPOINT, params=params)
  data = json.loads(response.text)
  values = data["observations"]
  df = pd.DataFrame(values)
  return df

In [8]:
tga = pull_data(TGA)
repo = pull_data(REPO)
bs = pull_data(BS)
sp = pull_data(SP500)

In [9]:
#format tga https://fred.stlouisfed.org/series/WTREGEN
def format_tga(tga):
  tga.loc[:,'date'] = pd.to_datetime(tga['date'])
  tga = tga[tga.value != '.']
  tga.loc[:, 'value'] = pd.to_numeric(tga['value'])
  tga_weekly = tga.groupby(pd.Grouper(key='date', freq='D'))
  tga_weekly_sum = tga_weekly.mean()
  tga_weekly_sum = tga_weekly_sum.reset_index()
  tga_weekly_sum = tga_weekly_sum[['date','value']]
  tga_weekly_sum.rename(columns={'value':'tga_value'}, inplace=True)
  tga_weekly_sum = tga_weekly_sum.fillna(method='ffill')
  return tga_weekly_sum

def format_bs(bs):
  bs['date'] = pd.to_datetime(bs['date'])
  bs.loc[:, 'value'] = pd.to_numeric(bs['value'])
  bs = bs.set_index('date',drop=True)
  bs_weekly_sum = bs.resample("D").mean()
  bs_weekly_sum = bs_weekly_sum.reset_index()
  bs_weekly_sum = bs_weekly_sum[['date','value']]
  bs_weekly_sum.loc[:, 'value'] *= 0.001 # move to billions
  bs_weekly_sum.rename(columns={'value':'bs_value'}, inplace=True)
  bs_weekly_sum = bs_weekly_sum.fillna(method='ffill')
  return bs_weekly_sum

def format_repo(repo):
  repo.loc[:,'date'] = pd.to_datetime(repo['date'])
  repo = repo[repo.value != '.']
  repo.loc[:, 'value'] = pd.to_numeric(repo['value'])
  repo_weekly = repo.groupby(pd.Grouper(key='date', freq='D'))
  repo_weekly_sum = repo_weekly.mean()
  repo_weekly_sum = repo_weekly_sum.reset_index()
  repo_weekly_sum = repo_weekly_sum[['date','value']] # final REPO dataframe
  repo_weekly_sum.rename(columns={'value':'repo_value'}, inplace=True)
  repo_weekly_sum = repo_weekly_sum.fillna(method='ffill')
  return repo_weekly_sum

tga_weekly_sum = format_tga(tga)
bs_weekly_sum = format_bs(bs)
repo_weekly_sum = format_repo(repo)

# format S&P500 df
sp = format_tga(sp)
sp.rename(columns={'tga_value':'sp_value'}, inplace=True)

<ipython-input-9-281fbc0e1f22>:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  tga.loc[:,'date'] = pd.to_datetime(tga['date'])
<ipython-input-9-281fbc0e1f22>:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  tga.loc[:, 'value'] = pd.to_numeric(tga['value'])
<ipython-input-9-281fbc0e1f22>:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tga_weekly_sum 

In [10]:
# get bitcoin price
def pull_bitcoin():

  # url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=max&interval=daily"
  url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=max&interval=daily"
  response = requests.get(url)
  data = response.json()
  data = data['prices']
  df = pd.DataFrame(data)
  df['date'] = df.iloc[:,0]
  df.loc[:,'date'] = pd.to_numeric(df['date'])

  df['btc_value'] = df.iloc[:,1]
  df.loc[:,'btc_value'] = pd.to_numeric(df['btc_value'])
  df.loc[:,'date'] = pd.to_datetime(df['date'], unit='ms')
  df = df[['date', 'btc_value']]

  df = df.groupby(pd.Grouper(key='date', freq='D'))
  df = df.mean()
  btc_df = df.reset_index()


  return btc_df

btc_df = pull_bitcoin()

In [11]:
#combine dataframes
combine_dfs = tga_weekly_sum.merge(bs_weekly_sum, on='date').merge(repo_weekly_sum, on='date').merge(btc_df, on='date').merge(sp, on='date')
#add net liquidity
combine_dfs['net_liq'] = combine_dfs['bs_value'] - combine_dfs['tga_value'] - combine_dfs['repo_value']
#add net liq divided by S&P 500
combine_dfs['ratio'] = combine_dfs['net_liq'] / combine_dfs['btc_value']

In [12]:
import plotly.express as px
fig = px.line(combine_dfs, x='date', y=['tga_value','bs_value','repo_value', 'net_liq', 'btc_value', 'sp_value'])

fig.show()

In [13]:
net_liq = combine_dfs.loc[combine_dfs['date'] > '2020-01-01']
fig = px.line(net_liq, x='date', y='net_liq')
fig.show()

In [14]:

fig_2 = px.line(net_liq, x='date',  y = 'ratio')

fig_2.show()

In [15]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=net_liq['date'], y=net_liq['net_liq'], name="net liquidity"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=net_liq['date'], y=net_liq['btc_value'], name="btc value"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Net Liquidity Versus Bitcoin"
)

# Set x-axis title
#fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Net Liquidity (B$)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Bitcoin</b> yaxis title", secondary_y=True)

fig.show()

In [16]:
# Create figure with secondary y-axis
fig2 = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig2.add_trace(
    go.Scatter(x=net_liq['date'], y=net_liq['net_liq'], name="net liquidity"),
    secondary_y=False,
)

fig2.add_trace(
    go.Scatter(x=net_liq['date'], y=net_liq['sp_value'], name="SP500 value"),
    secondary_y=True,
)

# Add figure title
fig2.update_layout(
    title_text="Net Liquidity Versus S&P500"
)

# Set x-axis title
#fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig2.update_yaxes(title_text="<b>Net Liquidity (B$)</b>", secondary_y=False)
fig2.update_yaxes(title_text="<b>S&P500</b>", secondary_y=True)

fig2.show()